# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

`See pdf report for answer.`

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = len(student_data)
n_features = len(student_data.columns) - 1
n_passed = len(student_data[student_data['passed'] == 'yes'])
n_failed = n_students - n_passed
grad_rate = float(n_passed) / float(n_students) * 100.
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
import sklearn.cross_validation as cv
X_train, X_test, y_train, y_test = cv.train_test_split(X_all, y_all, train_size=num_train, stratify=y_all)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

print "\nPass/Fail distrubution in train and test sets:"
print pd.Series(y_train).value_counts()
print pd.Series(y_test).value_counts()

Training set: 300 samples
Test set: 95 samples

Pass/Fail distrubution in train and test sets:
yes    201
no      99
Name: passed, dtype: int64
yes    64
no     31
Name: passed, dtype: int64


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [7]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)
    return end - start

# TODO: Choose a model, import it and instantiate an object
from sklearn import svm
clf = svm.SVC(kernel='rbf', C=1.0, gamma=0.01)

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.018


0.018409013748168945

In [8]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes'), end - start

train_f1_score, predict_time = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.012
F1 score for training set: 0.833333333333


In [9]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
F1 score for test set: (0.80519480519480513, 0.002966165542602539)


In [10]:
from collections import namedtuple
Score = namedtuple('Score', 'size train_time predict_time f1score_train f1score_test')

# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_time = train_classifier(clf, X_train, y_train)
    
    f1score_train, _ = predict_labels(clf, X_train, y_train)
    f1score_test, predict_time = predict_labels(clf, X_test, y_test)
    
    return Score(len(y_train), train_time, predict_time, f1score_train, f1score_test)
    

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
def learning_curve(clf):

    subsets = [100, 200, 300]
    scores = []
    for train_fraction in subsets:
        if train_fraction == 1.0 or train_fraction / len(y_train) == 1.0:
            xtr, ytr = X_train, y_train
        else:
            xtr, _, ytr, _ = cv.train_test_split(X_train, y_train, train_size=train_fraction)

        scores.append(train_predict(clf, xtr, ytr, X_test, y_test))
        
    results = pd.DataFrame(scores)
    print
    print results.to_latex(index=False)
    return results

clf = svm.SVC(kernel='rbf', C=1.0, gamma=0.01)
learning_curve(clf)


------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.002
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.006
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.010
Predicting labels using SVC...
Done!
Prediction time (secs): 0.009
Predicting labels using SVC...
Done!
Prediction time (secs): 0.004

\begin{tabular}{rrrrr}
\toprule
 size &  train\_time &  predict\_time &  f1score\_train &  f1score\_test \\
\midrule
  100 &    0.001937 &      0.001134 &       0.836601 &      0.797386 \\
  200 &    0.006036 &      0.003433 &       0.843658 &      

,size,train_time,predict_time,f1score_train,f1score_test
0,100,0.001937,0.001134,0.836601,0.797386
1,200,0.006036,0.003433,0.843658,0.802548
2,300,0.010464,0.003730,0.833333,0.805195


In [11]:
# TODO: Train and predict using two other models

from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=32, weights='uniform')
learning_curve(clf) 

------------------------------------------
Training set size: 100
Training KNeighborsClassifier...
Done!
Training time (secs): 0.003
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.006
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.014
------------------------------------------
Training set size: 200
Training KNeighborsClassifier...
Done!
Training time (secs): 0.002
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.014
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.008
------------------------------------------
Training set size: 300
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.025
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.006

\begin{tabular}{rrrrr}
\toprule
 size &  train\_time &  predict\_time &  f1score\_train &  f1sco

,size,train_time,predict_time,f1score_train,f1score_test
0,100,0.002566,0.013909,0.782609,0.810127
1,200,0.002431,0.007819,0.809816,0.815287
2,300,0.001159,0.006072,0.820833,0.812903


In [12]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_features=30, max_depth=4, min_samples_split=8, min_samples_leaf=1)
learning_curve(clf) 

------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.003
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001

\begin{tabular}{rrrrr}
\toprule
 size &  train\_time &  predict\_time &  f1sco

,size,train_time,predict_time,f1score_train,f1score_test
0,100,0.001149,0.000357,0.894410,0.818182
1,200,0.001343,0.000411,0.874576,0.800000
2,300,0.002917,0.001213,0.860412,0.811594


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [13]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn import grid_search
from sklearn.metrics import f1_score, make_scorer
        
clf = svm.SVC

param_grid = {
    'kernel':('linear', 'rbf'),
    'C':[0.2, 1, 2], 
    'gamma': ['auto', 0.1, 0.04, 0.02, 0.01, 0.008, 0.005]
}

gridclf = grid_search.GridSearchCV(clf(), param_grid,
                                   scoring=make_scorer(f1_score),
                                   refit=True, verbose=True)
gridclf.fit(X_train, y_train == 'yes')


# The gridsearch returs the mean score over a 3-Fold on the training set. 
# Here we train using the entire training set and evaluate on the test set. 
# At the end we show a dataframe of the results, sorted by the best F1 score on
# the test set. 
results = []
for result in gridclf.grid_scores_:
    dictout = result.parameters

    estimator = clf(**dictout)

    start = time.time()
    estimator.fit(X_train, y_train == 'yes')
    end = time.time()
    dictout['fit_time'] = end - start

    start = time.time()
    pred_ts = estimator.predict(X_test)
    end = time.time()
    dictout['predict_time'] = end - start

    dictout['test_score'] = f1_score(pred_ts, y_test == 'yes')
    dictout['cv_score'] = result.mean_validation_score

    # Here I will store number of yes/no predictions for the test set
    dictout['pred_pass'] = pred_ts.sum()
    dictout['pred_fail'] = -1.*(pred_ts-1).sum() 

    assert len(pred_ts) == dictout['pred_pass'] + dictout['pred_fail']
    results.append(dictout)

import pandas as pd
grid_search_df = pd.DataFrame(results)
grid_search_df['index'] = grid_search_df.index

grid_search_df.sort_values(by='test_score', ascending=False)

[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    1.0s
[Parallel(n_jobs=1)]: Done 126 out of 126 | elapsed:    3.5s finished


Fitting 3 folds for each of 42 candidates, totalling 126 fits


,C,cv_score,fit_time,gamma,kernel,pred_fail,pred_pass,predict_time,test_score,index
17,1.0,0.807337,0.014904,0.1,rbf,3,92,0.004977,0.820513,17
41,2.0,0.809009,0.010580,0.005,rbf,6,89,0.003396,0.810458,41
25,1.0,0.804864,0.014517,0.008,rbf,5,90,0.003691,0.805195,25
23,1.0,0.808145,0.013033,0.01,rbf,5,90,0.002570,0.805195,23
3,0.2,0.802395,0.014064,0.1,rbf,-0,95,0.003124,0.805031,3
5,0.2,0.802395,0.010235,0.04,rbf,-0,95,0.003251,0.805031,5
7,0.2,0.802395,0.010960,0.02,rbf,-0,95,0.003699,0.805031,7
11,0.2,0.802395,0.010429,0.008,rbf,-0,95,0.004842,0.805031,11
13,0.2,0.802395,0.016499,0.005,rbf,-0,95,0.005192,0.805031,13
1,0.2,0.802395,0.012584,auto,rbf,-0,95,0.004278,0.805031,1


In [14]:
with open('gridsearch.tex', 'w') as fout:
    grid_search_df.sort_values(by='test_score', ascending=False)\
    [['kernel', 'C', 'gamma', 'cv_score', 'test_score']].to_latex(fout, index=False)